In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yfinance as yf
!pip install dateparser
import dateparser

In [ ]:
inicio = '2022-01-01'
fim = '2024-02-01'

In [ ]:
def merge_yfin(ativos,data_inicio,data_fim):

  inicio = data_inicio
  fim = data_fim
  contador = 0
  df_ativos = pd.DataFrame(columns = ['Date','Adj Close'])
  for ativo in ativos:
    df_ativo = yf.download(ativo, start = inicio, end = fim)['Adj Close']
    df_ativo = df_ativo.to_frame().reset_index()
    df_ativo = df_ativo.rename(columns = {'Adj Close' : '{}'.format(ativo.replace(".SA", ""))})
    df_ativo['ano'] = pd.DatetimeIndex(df_ativo['Date']).year
    df_ativo['mes'] = pd.DatetimeIndex(df_ativo['Date']).month
    #df_ativo['dia'] = pd.DatetimeIndex(df_ativo['Date']).day
    df_ativo = df_ativo.groupby(by = ['ano','mes']).agg('mean').reset_index()
    if contador == 0:
      df_ativos = df_ativo
      contador += 1
    else:
      df_ativos = pd.merge(df_ativos, df_ativo,  how='left', left_on = ['ano','mes'], right_on = ['ano','mes'])
     # Criar coluna de data completa
    #df_ativos['dia']=1
    #print(df_ativos.info())
    #df_ativos['Data Completa'] = pd.to_datetime(df_ativos[['ano', 'mes','dia']])
    # Excluir as colunas de ano e mês
   # df_ativos.drop(['ano', 'mes'], axis=1, inplace=True)
  return df_ativos

In [ ]:
#Dados Bacen

def get_bc_dataset(CodSerie):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(CodSerie)
  raw_df =  pd.read_json(url)
  raw_df = raw_df.rename(columns={'valor': CodSerie})
  raw_df = trat_datas(raw_df,'data')
  raw_df = raw_df.groupby(by=["ano", "mes"], as_index=False).agg('mean')
  return raw_df

def merge_datasets(df1,df2,key):
  df = pd.merge(df1, df2,  how='left', left_on = key, right_on = key)
  return df

# Para ganharmos escalabilidade, a função get_pool realiza a função merge_datasets para uma lista de series do bacen a serem analisadas,
# ou seja, faz lago de dados (data lake) com os indicadores e kpis indicados (numero das series do bacem) em uma lista

def lago(lista_series_bc):

  df = pd.DataFrame(columns = ['ano','mes'])
  contador = 0
  for i in lista_series_bc:
    df_i = get_bc_dataset(i)
    if contador == 0:
      df = df_i
      contador += 1
    else:
      df = merge_datasets(df,df_i,['ano','mes'])
  return df

def trat_datas(df,coluna_data):
  df['ano'] = df.apply(lambda _: '', axis=1)
  df['mes'] = df.apply(lambda _: '', axis=1)

  for p,i in enumerate (df[coluna_data]):
    i = dateparser.parse(i)
    df['mes'].iloc[p] = df[coluna_data].iloc[p][3:5]
    df['ano'].iloc[p] = df[coluna_data].iloc[p][-4:]

  return df

In [ ]:
#Dados Yahoo Finance
#'BTC-USD',
lista_Yfinance = ['^BVSP','USDBRL=X','^GSPC']

df_yF = merge_yfin(lista_Yfinance, inicio, fim)

[*********************100%%**********************]  1 of 1 completed
<ipython-input-156-d1119194effe>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ativo = df_ativo.groupby(by = ['ano','mes']).agg('mean').reset_index()
[*********************100%%**********************]  1 of 1 completed
<ipython-input-156-d1119194effe>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ativo = df_ativo.groupby(by = ['ano','mes']).agg('mean').reset_index()
[*********************100%%**********************]  1 of 1 completed
<ipython-input-156-d1119194effe>:14: FutureWarning: The default value of numeric_only in Data

In [ ]:
df_yF = df_yF.rename(columns = {'^BVSP' : 'Ibovespa', 'USDBRL=X' : 'USD/BRL', '^GSPC' : 'SP500'})

# Convertendo as colunas 'ano' e 'mes' para inteiros
df_yF['ano'] = df_yF['ano'].astype(int)
df_yF['mes'] = df_yF['mes'].astype(int)

# Adicionando uma coluna 'dia' com o valor 1 para todos os registros
df_yF['dia'] = 1

# Convertendo as colunas 'ano', 'mes' e 'dia' para o formato datetime
df_yF['Data'] = df_yF.apply(lambda x: pd.to_datetime(f"{int(x['ano'])}-{int(x['mes'])}-{int(x['dia'])}", format='%Y-%m-%d'), axis=1)

# Drop the columns that are no longer needed
df_yF.drop(columns=['ano', 'mes', 'dia'], inplace=True)

# Define the columns to keep as identifiers (non-melted columns)
#id_vars = ['Data']

# Use pd.melt() to transform columns into rows
#df_yF = pd.melt(df_yF, id_vars=id_vars, var_name='Ativo', value_name='Valor')

In [ ]:
#Dados Bacen
lista_bacen = [4390,189,433]
#lista_bacen = [4390]
df = lago(lista_bacen)


df['ano'] = df['ano'].astype(int)
df['mes'] = df['mes'].astype(int)
df = df.rename(columns={
    4390:'Selic',
    189:'IGPM',
    433:'IPCA',
})

<ipython-input-157-16023c5a8a2a>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mes'].iloc[p] = df[coluna_data].iloc[p][3:5]
<ipython-input-157-16023c5a8a2a>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano'].iloc[p] = df[coluna_data].iloc[p][-4:]
<ipython-input-157-16023c5a8a2a>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  raw_df = raw_df.groupby(by=["ano", "mes"], as_index=False).agg('mean')
<ipython-input-1

In [ ]:
# Convertendo as colunas 'ano' e 'mes' para inteiros
df['ano'] = df['ano'].astype(int)
df['mes'] = df['mes'].astype(int)

# Adicionando uma coluna 'dia' com o valor 1 para todos os registros
df['dia'] = 1

# Convertendo as colunas 'ano', 'mes' e 'dia' para o formato datetime
df['Data'] = df.apply(lambda x: pd.to_datetime(f"{int(x['ano'])}-{int(x['mes'])}-{int(x['dia'])}", format='%Y-%m-%d'), axis=1)

# Drop the columns that are no longer needed
df.drop(columns=['ano', 'mes', 'dia'], inplace=True)
df = df.query('Data >= @inicio and Data <=@fim')
df = df.dropna()

In [ ]:
df = pd.merge(df, df_yF, on='Data', how='left')

colunas = list(df.columns)
colunas.remove('Data')
colunas.insert(0, 'Data')
df = df[colunas]

In [ ]:
df['SP500/BRL'] = df['SP500'] * df['USD/BRL']

In [ ]:
#df

,Data,Selic,IGPM,IPCA,Ibovespa,USD/BRL,SP500,SP500/BRL
0,2022-01-01,0.73,1.82,0.54,106724.380952,5.537409,4573.815454,25327.087115
1,2022-02-01,0.76,1.83,1.01,112851.263158,5.197918,4435.980494,23057.861951
2,2022-03-01,0.93,1.74,1.62,115150.045455,4.987289,4391.265264,21900.507189
3,2022-04-01,0.83,1.41,1.06,115022.052632,4.732981,4391.295996,20783.920418
4,2022-05-01,1.03,0.52,0.47,107809.227273,4.960875,4040.360003,20043.721243
5,2022-06-01,1.02,0.59,0.67,103746.571429,5.032583,3898.946684,19621.772795
6,2022-07-01,1.03,0.21,-0.68,99108.571429,5.371471,3911.729492,21011.743200
7,2022-08-01,1.17,-0.70,-0.36,110061.826087,5.142541,4158.563084,21385.580272
8,2022-09-01,1.07,-0.95,-0.29,110720.904762,5.224400,3850.520496,20116.659281
9,2022-10-01,1.02,-0.97,0.59,115599.950000,5.263060,3726.050956,19610.429497


In [ ]:
df.to_parquet('Indicadores.parquet')